In [1]:
import sys
sys.path.append('../')

import os
import os.path as osp
import numpy as np
import pickle

from dataloading.imports import GESTURE_NAMES
from dataloading.visualize import visualize_samples
import dataloading.transform as T

#### Prepare data and train the model
We load the previously packaged data and transform it based on the hyper-parameters.
Most importantly, we fit a polynomial to each axis of a given time series, in order to transform it to a fixed-sized vector that can be fed into the classifier.

In [2]:
NUM_INTEGRATE = 1
NORM_VECTORS = False 
CENTER_SCALE = None # None: not at all, True: norm per axis, False: Overall
POLY_DEGREES = 2
AXES = [0, 1, 2]
NORM_COEFS = False


with open('../data/uwave/uwave.pkl', 'rb') as f:
    data = pickle.load(f)

for i in range(NUM_INTEGRATE):
    data = T.integrate_acceleration(data)

if NORM_VECTORS:
    data = T.normalize_vectors(data)

if CENTER_SCALE is not None:
    data = T.center_scale(data, scale_per_axis=CENTER_SCALE)  

for key in data:
    for i in range(len(data[key])):
        data[key][i] = data[key][i][:, AXES]

# Fit polynomials to the time-series and aggregate the data to X and y
data_poly = T.polyfit_timeseries(data, deg=POLY_DEGREES)
X, y = T.aggregate_data(data_poly)
X_train, y_train, X_test, y_test = T.random_split(X, y)

if NORM_COEFS:
    X_train, mean, std = T.normalize_X(X_train)
    X_test, _, _ = T.normalize_X(X_test, mean=mean, std=std)

/home/imanox/.local/lib/python3.6/site-packages/numpy/polynomial/polynomial.py:1350: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)


The model is trained on the transformed data, using appropriate model hyper-parameters.

In [3]:
from models.logistic_regression import LogisticRegression

classifier = LogisticRegression(lr=0.05, reg=0.001, max_iter=1000, tol=0.5)
classifier.fit(X_train, y_train, verbose=False)

pred_train = classifier.predict(X_train)
pred_test = classifier.predict(X_test)

train_acc = np.mean(pred_train == y_train)
test_acc = np.mean(pred_test == y_test)
print(f'Acc: {train_acc:0.2f} / {test_acc:0.2f}')    

Acc: 0.78 / 0.79


#### Package the model for deployment
In this case, we can package the model with weights using pickle.

In [ ]:
with open('../checkpoints/LogisticRegression.pkl', 'wb') as f:
    pickle.dump(classifier, f)